In [ ]:
#No57(特徴量の重みの確認)
import pandas as pd
import string
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from google.colab import drive
drive.mount('/content/drive')
!wget https://archive.ics.uci.edu/static/public/359/news+aggregator.zip
!unzip news+aggregator.zip
df = pd.read_csv("./newsCorpora.csv",sep="\t",header=None,names=["ID", "TITLE", "URL", "PUBLISHER", "CATEGORY", "STORY", "HOSTNAME", "TIMESTAMP"])
#「PUBLISHER」の行から、”Reuters”, “Huffington Post”, “Businessweek”, “Contactmusic.com”, “Daily Mail”の事例（記事）のみを抽出する．
#isinはあくまでbool値を返すことに注意(Trueが抽出される)
df = df[df["PUBLISHER"].isin(["Reuters", "Huffington Post", "Businessweek", "Contactmusic.com", "Daily Mail"])]
#「TITLE」と「CATEGORY」の列を抽出する．
df = df[["TITLE", "CATEGORY"]]
#学習、検証、評価データに分割する(分割したいもの、割合、shuffleはTrueがデフォルト)
#まず0.9:0.1で分ける
train, test = train_test_split(df, test_size=0.2)
#次に0.2を半分にする(検証、評価データを0.1にする)
test, valid = train_test_split(test, test_size=0.5)

In [ ]:
def preprosessing(text):
    #string.punctuation 「'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'」のこと
    table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    #maketransで作成したtableを文字列を変換する
    text = text.translate(table)
    #小文字にする
    text = text.lower()
    #正規表現を利用するために、compileを用いてpatternを作成する
    pattern = re.compile('[0-9]+')
    #正規表現にマッチした部分に0を代入
    text = re.sub(pattern, '0', text)

    return text

#データの連結、前処理
df = pd.concat([train, valid, test], axis = 0)
#もとのindexを削除
df.reset_index(drop=True, inplace=True)

df['TITLE'] = df['TITLE'].map(lambda x: preprosessing(x))
#データの連結、前処理
df = pd.concat([train, valid, test], axis = 0)
df.reset_index(drop=True, inplace=True)
#map関数　シーケンスの構成要素すべてに対して、ある関数の処理を行わせるという高階関数
#lambda関数を用いて、xを引数として、preprosessing関数を呼び出す
df['TITLE'] = df['TITLE'].map(lambda x: preprosessing(x))
vec_tfidf = TfidfVectorizer() #TfidfVectorizerのインスタンス生成
data = vec_tfidf.fit_transform(df['TITLE'])
data = pd.DataFrame(data.toarray(), columns = vec_tfidf.get_feature_names_out())
#整数除算をして、dataを分割する。
split_point1 = int(len(data)//3)
split_point2 = int(split_point1 * 2)
#学習、検証、評価データ
x_train = data[:split_point1]
x_valid = data[split_point1:split_point2]
x_test = data[split_point2:]
#学習、検証、評価等別
y_data = df['CATEGORY']
y_train = y_data[:split_point1]
y_valid = y_data[split_point1:split_point2]
y_test = y_data[split_point2:]

In [ ]:
from sklearn.linear_model import LogisticRegression
#ロジスティック回帰モデルのインスタンス生成
model = LogisticRegression()
#訓練の実行(学習データ、ラベル(正解))
model.fit(x_train, y_train)

In [ ]:
# 正解率計算用のメソッド
from sklearn.metrics import accuracy_score
y_pred = model.predict(x_valid)
#訓練データとテストデータを用いて予測する
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

In [ ]:
import numpy as np

features = x_train.columns.values  # 学習データの特徴量
#LogisticRegression.classes  分類したカテゴリのラベルを取得する
#LogisticRegression.coef     ロジスティック回帰の係数（＝特徴量ごとの重み）を取得する。
class_name, coef = model.classes_[0], model.coef_[0]
#10位まで取得
index = [i for i in range(1, 11)]
#任意の行または列を基準にソートする場合は、np.argsort()を使う
#afgsortは昇順がデフォルトだが、-を使うことにより、降順にできる。
top_10 = features[np.argsort(-coef)[:10]]
worst_10 = features[np.argsort(coef)[:10]]

df_top_10 = pd.DataFrame(top_10, columns=[f'重みの高い特徴量トップ10（クラス名: {class_name}）'], index = index)
df_worst_10 = pd.DataFrame(worst_10, columns=[f'重みの低い特徴量トップ10（クラス名: {class_name}）'], index= index)

print(df_top_10)
print(df_worst_10)

